In [88]:
import os
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad

## Bit FLipper

In [89]:
def bit_flipper(file_input, file_output):
    with open (file_input, "rb") as f :
        data = bytearray(f.read())
        data [50] ^= 0x01

    # cambia un bit dentro de C_3 ( ciphertext )
    with open (file_output, "wb") as f :
        f.write(data)

## Block Swaper

In [90]:
def swap_blocks(input_file, output_file, block_size=16, i=2, j=3):
    """
    Intercambia los bloques Ci y Cj de un archivo cifrado en formato:
    C0 || C1 || C2 ... Cn
    block_size normalmente = 16 (AES).
    """

    with open(input_file, "rb") as f:
        data = bytearray(f.read())

    #print(data)
    # Calcular offset en bytes de cada bloque
    start_i, end_i = i * block_size, (i+1) * block_size
    start_j, end_j = j * block_size, (j+1) * block_size

    # Intercambiar bloques
    block_i = data[start_i:end_i]
    block_j = data[start_j:end_j]
    data[start_i:end_i], data[start_j:end_j] = block_j, block_i

    with open(output_file, "wb") as f:
        f.write(data)

    print(f"Intercambiados C{i} y C{j} en {input_file}, guardado en {output_file}")


## Key Generator

In [91]:
# Función para generar una clave a partir de una contraseña usando SHA256
def getKey(password):
        # Crear un objeto hash SHA256
        hasher = SHA256.new(password.encode('utf-8'))
        # Devolver el resumen del hash (la clave)
        return hasher.digest()

## Comparador de files

In [92]:
def compare_files_per_byte(file1, file2, block_size=16, max_diffs=64):
    """
    Compara dos archivos byte por byte y muestra diferencias.
    Agrupa por bloques para ver qué zonas se dañaron.
    """
    with open(file1, "rb") as f:
        data1 = f.read()
    with open(file2, "rb") as f:
        data2 = f.read()

    for i, (b1, b2) in enumerate(zip(data1, data2)):
        if b1 != b2:
            block_index = i // block_size
            print(f"Diferencia en byte {i} (bloque {block_index}): {b1} != {b2}")
            max_diffs -= 1
            if max_diffs == 0:
                print("...más diferencias ocultas...")
                break



In [93]:
def compare_files_per_block(file1, file2, block_size=16):
    """
    Compara dos archivos y reporta únicamente los bloques que están corrompidos.
    """
    with open(file1, "rb") as f:
        data1 = f.read()
    with open(file2, "rb") as f:
        data2 = f.read()



    corrupted_blocks = set()

    # Comparar byte por byte y registrar bloques corrompidos
    for i in range(min(len(data1), len(data2))):
        if data1[i] != data2[i]:
            block_index = i // block_size + 1
            corrupted_blocks.add(block_index)

    # Mostrar resultados
    if corrupted_blocks:
        sorted_blocks = sorted(corrupted_blocks)
        print(f"Bloques corrompidos: {sorted_blocks}")
        print(f"Total de bloques afectados: {len(corrupted_blocks)}")
    else:
        print("Los archivos son idénticos - no hay bloques corrompidos")

# **Funciones de Encriptado y Desencriptado**

In [94]:
def encrypt(mode, key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # Nombre del archivo de salida cifrado
        outputFile = filename+".enc"
        # Obtener el tamaño del archivo original y rellenar con ceros a la izquierda para que tenga 16 bytes
        filesize = str(os.path.getsize(filename)).zfill(16)
        # Generar un vector de inicialización (IV) aleatorio de 16 bytes

        # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
        if mode == "ECB":
            encryptor= AES.new(key, AES.MODE_ECB)
        elif mode == "CBC":
            IV = Random.new().read(16)
            encryptor= AES.new(key, AES.MODE_CBC, IV)
        else:
            print("Error de mode")
            return

        # Abrir el archivo de entrada en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Escribir el tamaño del archivo original codificado en utf-8
                        outfile.write(filesize.encode('utf-8'))


                        # Escribir el vector de inicialización (IV)
                        if mode == "CBC":
                            outfile.write(IV)


                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break
                                # Si el tamaño del chunk no es múltiplo de 16, rellenar con bytes nulos
                                elif len(chunk)%16 != 0:
                                        chunk=pad(chunk, 16)

                                # Cifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(encryptor.encrypt(chunk))

# Función para descifrar un archivo
def decrypt(mode, key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # ruta del archivo de salida descifrado (eliminar la extensión .enc)
        outputFile = filename[:-4]

        # Abrir el archivo de entrada cifrado en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Leer el tamaño del archivo original (los primeros 16 bytes) y convertirlo a entero
                filesize = int(infile.read(16))

                # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
                if mode == "ECB":
                    decryptor= AES.new(key, AES.MODE_ECB)
                elif mode == "CBC":
                    IV = infile.read(16)
                    decryptor= AES.new(key, AES.MODE_CBC, IV)
                else:
                    print("Error de mode")
                    return

                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break

                                # Descifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(decryptor.decrypt(chunk))

                        # Truncar el archivo de salida al tamaño original
                        outfile.truncate(filesize)

## Parte 3 - Intercambio de Bloques

In [95]:
encrypt("CBC", getKey("1234"), "test.pdf")
swap_blocks("test.pdf.enc", "Resultados/CBC - Block Swap/test_swap.pdf.enc", 16, 3, 5)
decrypt("CBC", getKey("1234"), "Resultados/CBC - Block Swap/test_swap.pdf.enc")
compare_files_per_byte("test.pdf", "Resultados/CBC - Block Swap/test_swap.pdf")
print("=====================================================================================================================")
compare_files_per_block("test.pdf", "Resultados/CBC - Block Swap/test_swap.pdf")

Intercambiados C3 y C5 en test.pdf.enc, guardado en Resultados/CBC - Block Swap/test_swap.pdf.enc
Diferencia en byte 16 (bloque 1): 32 != 83
Diferencia en byte 17 (bloque 1): 48 != 29
Diferencia en byte 18 (bloque 1): 32 != 201
Diferencia en byte 19 (bloque 1): 111 != 17
Diferencia en byte 20 (bloque 1): 98 != 219
Diferencia en byte 21 (bloque 1): 106 != 96
Diferencia en byte 22 (bloque 1): 10 != 137
Diferencia en byte 23 (bloque 1): 60 != 18
Diferencia en byte 24 (bloque 1): 60 != 132
Diferencia en byte 25 (bloque 1): 10 != 174
Diferencia en byte 26 (bloque 1): 47 != 1
Diferencia en byte 27 (bloque 1): 76 != 94
Diferencia en byte 28 (bloque 1): 101 != 122
Diferencia en byte 29 (bloque 1): 110 != 2
Diferencia en byte 30 (bloque 1): 103 != 151
Diferencia en byte 31 (bloque 1): 116 != 36
Diferencia en byte 32 (bloque 2): 104 != 14
Diferencia en byte 33 (bloque 2): 32 != 249
Diferencia en byte 34 (bloque 2): 49 != 42
Diferencia en byte 35 (bloque 2): 48 != 46
Diferencia en byte 36 (bloque

## Pruebas en ECB

### Bit FLipping (ECB)

In [96]:
encrypt("ECB", getKey("1234"), "test.pdf")
bit_flipper("test.pdf.enc", "Resultados/ECB - Bit flip/test_flip.pdf.enc")
decrypt("ECB", getKey("1234"), "Resultados/ECB - Bit flip/test_flip.pdf.enc")

compare_files_per_byte("test.pdf", "Resultados/ECB - Bit flip/test_flip.pdf")
print("=====================================================================================================================")
compare_files_per_block("test.pdf", "Resultados/ECB - Bit flip/test_flip.pdf")

Diferencia en byte 32 (bloque 2): 104 != 168
Diferencia en byte 33 (bloque 2): 32 != 103
Diferencia en byte 34 (bloque 2): 49 != 8
Diferencia en byte 35 (bloque 2): 48 != 55
Diferencia en byte 36 (bloque 2): 49 != 143
Diferencia en byte 37 (bloque 2): 51 != 0
Diferencia en byte 38 (bloque 2): 32 != 112
Diferencia en byte 39 (bloque 2): 32 != 5
Diferencia en byte 40 (bloque 2): 32 != 90
Diferencia en byte 41 (bloque 2): 32 != 95
Diferencia en byte 42 (bloque 2): 32 != 30
Diferencia en byte 43 (bloque 2): 32 != 235
Diferencia en byte 44 (bloque 2): 10 != 188
Diferencia en byte 45 (bloque 2): 47 != 154
Diferencia en byte 46 (bloque 2): 70 != 114
Diferencia en byte 47 (bloque 2): 105 != 100
Bloques corrompidos: [3]
Total de bloques afectados: 1


### Code swapping (ECB)

In [97]:
encrypt("ECB", getKey("1234"), "test.pdf")
swap_blocks("test.pdf.enc", "Resultados/ECB - Block Swap/test_swap.pdf.enc", 16, 3, 5)
decrypt("ECB", getKey("1234"), "Resultados/ECB - Block Swap/test_swap.pdf.enc")
compare_files_per_byte("test.pdf", "Resultados/ECB - Block Swap/test_swap.pdf")
print("=====================================================================================================================")
compare_files_per_block("test.pdf", "Resultados/ECB - Block Swap/test_swap.pdf")

Intercambiados C3 y C5 en test.pdf.enc, guardado en Resultados/ECB - Block Swap/test_swap.pdf.enc
Diferencia en byte 32 (bloque 2): 104 != 101
Diferencia en byte 33 (bloque 2): 32 != 10
Diferencia en byte 34 (bloque 2): 49 != 62
Diferencia en byte 35 (bloque 2): 48 != 62
Diferencia en byte 36 (bloque 2): 49 != 10
Diferencia en byte 37 (bloque 2): 51 != 115
Diferencia en byte 38 (bloque 2): 32 != 116
Diferencia en byte 39 (bloque 2): 32 != 114
Diferencia en byte 40 (bloque 2): 32 != 101
Diferencia en byte 41 (bloque 2): 32 != 97
Diferencia en byte 42 (bloque 2): 32 != 109
Diferencia en byte 43 (bloque 2): 32 != 10
Diferencia en byte 44 (bloque 2): 10 != 120
Diferencia en byte 45 (bloque 2): 47 != 218
Diferencia en byte 46 (bloque 2): 70 != 189
Diferencia en byte 47 (bloque 2): 105 != 87
Diferencia en byte 64 (bloque 4): 101 != 104
Diferencia en byte 65 (bloque 4): 10 != 32
Diferencia en byte 66 (bloque 4): 62 != 49
Diferencia en byte 67 (bloque 4): 62 != 48
Diferencia en byte 68 (bloque